# Introduction to Differential Forwarding Analysis
Intro sketch:
* Network engineers frequently have to make changes to forwarding behavior: add new routes, opening or closing certain flows, moving flows around, etc.
* These changes are hard to get right, and hard to validate.
* This notebook will show how batfish can help validate changes to network forwarding.
* Differential analyses answer questions about and/or validate changes between two snapshots of the network.

In [1]:
# Import packages and load questions
%run startup.py

/Users/matt/.pyenv/versions/3.6.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matt/src/pybatfish/pybatfish/client/commands.py:50: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


## Example 1
Intro: describe the network and the change we want to make. The network topology is AS2 from the example network. The network is overprovisioned with failover redundancy for the core router. All traffic is routed through `as2core1`, but will automatically switch to use `as2core2` in case of a failure or during maintenance.

In this example, we want to shift traffic to `as2core1` for maintenance.  To do that we'll cost out `as2core1`.

### Step 1: Test current behavior
Before beginning, let's check that the network is working as expected. First we load our snapshot into batfish.

In [2]:
EX1_NETWORK_NAME = "differential-example1"
EX1_BASE_NAME = "base"
EX1_BASE_PATH = "networks/differential-ex1-base"

bf_set_network(EX1_NETWORK_NAME)
bf_init_snapshot(EX1_BASE_PATH, name=EX1_BASE_NAME, overwrite=True)

'base'

In this step, we'll use the `reachability` question to validate that all traffic is currently being routed through `as2core1`. To do this, we'll tell batfish to search for flows that transit `as2core2`. The network is working as expected if no such flows are found.

We run the `reachability` question two `PathConstraints` parameters: the parameter `transitLocations="as2core2"` says to look for flows that transit `as2core2`. The parameter `startLocation=".* - as2core2"` means to search all flows that start *anywhere* in the network except `as2core2`. The parameter `actions="SUCCESS,FAILURE"` means batfish should return any flows it finds, whether they are delivered successfully or dropped along the way (the default is to only return successful flows). 

In [3]:
# Some external traffic to hosts is routed through as2core1
answer = bfq.reachability(
    pathConstraints=PathConstraints(startLocation="enter(as2border.*)", transitLocations="as2core1"),
    headers = HeaderConstraints(dstIps="ofLocation(host.*)"),
).answer(snapshot=EX1_BASE_NAME)
display_html(answer.frame())

,Flow,Traces,TraceCount
0,Src IP: 10.12.11.0Src Port: 0Dst IP: 2.128.0.101Dst Port: 22IP Protocol: TCPStart Location: as2border1 interface=GigabitEthernet0/0,"ACCEPTED1. node: as2border1 RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet2/0)5. node: host1 RECEIVED(eth0: filter::INPUT) ACCEPTED(InboundStep)",1
1,Src IP: 2.12.11.0Src Port: 0Dst IP: 2.128.0.101Dst Port: 22IP Protocol: TCPStart Location: as2border1 interface=GigabitEthernet1/0,"ACCEPTED1. node: as2border1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet2/0)5. node: host1 RECEIVED(eth0: filter::INPUT) ACCEPTED(InboundStep)",1
2,Src IP: 2.12.12.0Src Port: 0Dst IP: 2.128.0.101Dst Port: 22IP Protocol: TCPStart Location: as2border1 interface=GigabitEthernet2/0,"ACCEPTED1. node: as2border1 RECEIVED(GigabitEthernet2/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet2/0)5. node: host1 RECEIVED(eth0: filter::INPUT) ACCEPTED(InboundStep)",1
3,Src IP: 10.23.21.0Src Port: 0Dst IP: 2.128.0.101Dst Port: 22IP Protocol: TCPStart Location: as2border2 interface=GigabitEthernet0/0,"ACCEPTED1. node: as2border2 RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet2/0)5. node: host1 RECEIVED(eth0: filter::INPUT) ACCEPTED(InboundStep)",1
4,Src IP: 2.12.22.0Src Port: 0Dst IP: 2.128.0.101Dst Port: 22IP Protocol: TCPStart Location: as2border2 interface=GigabitEthernet1/0,"ACCEPTED1. node: as2border2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) FORWARDE

In [5]:
# No traffic to the hosts is routed through as2core2
answer = bfq.reachability(
    pathConstraints=PathConstraints(startLocation="enter(as2border.*)", transitLocations="as2core2"),
    headers = HeaderConstraints(dstIps="ofLocation(host.*)")
).answer(snapshot=EX1_BASE_NAME)
display_html(answer.frame())

,Flow,Traces,TraceCount


Great! It's always comforting to validate that the network is running as expected and desired. 

### Step 2: Author the change
Next, we'll cost out `as2core1`, causing traffic to route through `as2core2`. We'll implement this change offline in a new snapshot, and validate that the change won't affect end-to-end reachability. Below you can see the configuration changes we're going to make: adjust some ospf costs and **TODO do something that I guess has to do with BGP?**

```
$ diff -r networks/differential-ex1-base networks/differential-ex1-change
diff -r networks/differential-ex1-base/configs/as2core1.cfg networks/differential-ex1-change/configs/as2core1.cfg
67a68
>  ip ospf cost 500
71a73
>  ip ospf cost 500
76a79
>  ip ospf cost 500
81a85
>  ip ospf cost 500
111a116
>   neighbor as2 shutdown
```
### Step 3: Validate the change has the intended effect
Before we deploy the change to the network, let's verify that it is correct. The requirements for the change are:
1. No traffic is routed through `as2core1`.
1. End-to-end network behavior is unaffected.

Each of these requirements is essential. **TODO** we can say more to emphasize why each requirement is essential.

In [29]:
EX1_CHANGE_NAME = "change"
EX1_CHANGE_PATH = "networks/differential-ex1-change"

bf_init_snapshot(EX1_CHANGE_PATH, name=EX1_CHANGE_NAME, overwrite=True)

'change'

#### Requirement 1: No traffic is routed through `as2core1`.
We repeat our previous query, this time replacing `as2core2` with `as2core1`, since we want to ensure that now `as2core1` is never transited.

In [30]:
# No traffic is routed through as2core1
answer = bfq.reachability(
    pathConstraints = PathConstraints(startLocation="enter(as2border.*)", transitLocations="as2core1"),
    headers = HeaderConstraints(dstIps="ofLocation(host.*)"),
    actions = "SUCCESS,FAILURE"
).answer(snapshot=EX1_CHANGE_NAME)
display_html(answer.frame())

,Flow,Traces,TraceCount


In [31]:
# No traffic is routed through as2core1
answer = bfq.reachability(
    pathConstraints = PathConstraints(startLocation=".* - as2core.*", transitLocations="as2core1"),
    headers = HeaderConstraints(dstIps="ofLocation(.* - as2core.*)", srcIps="0.0.0.0/0"),
    actions = "SUCCESS,FAILURE"
).answer(snapshot=EX1_CHANGE_NAME)
display_html(answer.frame())

,Flow,Traces,TraceCount


Great! We've verified that our change does successfully cost out `as2core1`. 

#### Requirement 2: End-to-end network behavior is unchanged.
Before deploying the change to the network, let's make sure that the change does not affect end-to-end network behavior. We can do this by comparing our two snapshots using the `differentialReachability` question. This question will return flows that are successfully delivered in either snapshot but not the other.

In this example, we search for flows that are successfully delivered before we costed-out `as2core1` but not after. To exclude flows that start or end at `as2core1` itself, we use the parameter `startLocation=".* - as2core1"` that says to find flows that *do not* start at `as2core1` and the parameter `dstIps="ofLocation(.* - as2core1)"` that says to find flows addressed to *anywhere other than* `as2core1`.

In [33]:
answer = bfq.differentialReachability(
    pathConstraints=PathConstraints(startLocation="enter(as2border.*)"),
    headers=HeaderConstraints(dstIps="ofLocation(host.*)")
).answer(
    snapshot=EX1_CHANGE_NAME, 
    reference_snapshot=EX1_BASE_NAME)
display_html(answer.frame())

,Flow,Base_Traces,Base_TraceCount,Delta_Traces,Delta_TraceCount
0,Src IP: 10.12.11.0Src Port: 0Dst IP: 2.128.0.101Dst Port: 22IP Protocol: TCPStart Location: as2border1 interface=GigabitEthernet0/0,"NULL_ROUTED1. node: as2border1 RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: static [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) NULL_ROUTED(null_interface)",1,"ACCEPTED1. node: as2border1 RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet2/0)5. node: host1 RECEIVED(eth0: filter::INPUT) ACCEPTED(InboundStep)",1
1,Src IP: 2.12.11.0Src Port: 0Dst IP: 2.128.0.101Dst Port: 22IP Protocol: TCPStart Location: as2border1 interface=GigabitEthernet1/0,"NULL_ROUTED1. node: as2border1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: static [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) NULL_ROUTED(null_interface)",1,"ACCEPTED1. node: as2border1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet2/0)5. node: host1 RECEIVED(eth0: filter::INPUT) ACCEPTED(InboundStep)",1
2,Src IP: 2.12.12.0Src Port: 0Dst IP: 2.128.0.101Dst Port: 22IP Protocol: TCPStart Location: as2border1 interface=GigabitEthernet2/0,"NULL_ROUTED1. node: as2border1 RECEIVED(GigabitEthernet2/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: static [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) NULL_ROUTED(null_interface)",1,"ACCEPTED1. node: as2border1 RECEIVED(GigabitEthernet2/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: connected [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) TRANSMITTED(GigabitEthernet2/0)5. node: host1 RECEIVED(eth0: filter::INPUT) ACCEPTED(InboundStep)",1
3,Src IP: 10.23.21.0Src Port: 0Dst IP: 2.128.0.101Dst Port: 22IP Protocol: TCPStart Location: as2border2 interface=GigabitEthernet0/0,"NULL_ROUTED1. node: as2border2 RECEIVED(GigabitEthernet0/0: OUTSIDE_TO_INSIDE) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core2 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: static [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) NULL_ROUTED(null_interface)",

In [34]:
answer = bfq.differentialReachability(
    pathConstraints=PathConstraints(startLocation=".* - as2core.*"),
    headers=HeaderConstraints(dstIps="ofLocation(.* - as2core.*)", srcIps="0.0.0.0/0")
).answer(
    snapshot=EX1_CHANGE_NAME, 
    reference_snapshot=EX1_BASE_NAME)
display_html(answer.frame())

,Flow,Base_Traces,Base_TraceCount,Delta_Traces,Delta_TraceCount
0,Src IP: 0.0.0.0Src Port: 0Dst IP: 2.128.0.1Dst Port: 0IP Protocol: HOPOPTStart Location: as2border1,"NULL_ROUTED1. node: as2border1 ORIGINATED(default) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: static [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) NULL_ROUTED(null_interface)",1,"ACCEPTED1. node: as2border1 ORIGINATED(default) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core1 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1
1,Src IP: 0.0.0.0Src Port: 0Dst IP: 2.128.0.1Dst Port: 0IP Protocol: HOPOPTStart Location: as2border2,"NULL_ROUTED1. node: as2border2 ORIGINATED(default) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet1/0)2. node: as2core2 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: static [Network: 2.128.0.0/24, Next Hop IP:AUTO/NONE(-1l)]) NULL_ROUTED(null_interface)",1,"ACCEPTED1. node: as2border2 ORIGINATED(default) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)2. node: as2core1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ibgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet3/0)3. node: as2dist2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: bgp [Network: 2.128.0.0/24, Next Hop IP:2.34.201.4]) TRANSMITTED(GigabitEthernet2/0)4. node: as2dept1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)",1


Unfortunately we did find some changes in end-to-end behavior, indicating that there has been some drift between `as2core1` and `as2core2` over time. There are two flows traffic that are delivered before costing out `as2core1` but fails after, demonstrating a loss of availability.

The results include detailed traces of the flows through the network, which helps us diagnose the problem: **TODO**
- Currently we have a null route.
- Could instead have an ACL.

We fix this bug in a new version of the changed snapshot, and then upload the fixed change snapshot and repeat the validation steps: 

In [35]:
EX1_CHANGE_FIXED_NAME = "change-fixed"
EX1_CHANGE_FIXED_PATH = "networks/differential-ex1-change-fixed"

bf_init_snapshot(EX1_CHANGE_FIXED_PATH, name=EX1_CHANGE_FIXED_NAME, overwrite=True)

'change-fixed'

In [42]:
# Requirement 1: No traffic is routed through as2core1.
answer = bfq.reachability(
    pathConstraints = PathConstraints(startLocation="enter(as2border.*)", transitLocations="as2core1"),
    headers = HeaderConstraints(dstIps="ofLocation(host.*)", srcIps="0.0.0.0/0"),
    actions = "SUCCESS,FAILURE"
).answer(snapshot=EX1_CHANGE_FIXED_NAME)
display_html(answer.frame())

answer = bfq.reachability(
    pathConstraints = PathConstraints(startLocation=".* - as2core1", transitLocations="as2core1"),
    headers = HeaderConstraints(srcIps = "0.0.0.0/0"),
    actions = "SUCCESS,FAILURE"
).answer(snapshot=EX1_CHANGE_FIXED_NAME)
display_html(answer.frame())

,Flow,Traces,TraceCount


,Flow,Traces,TraceCount


In [44]:
# Requirement 2: End-to-end network behavior is unchanged.
answer = bfq.differentialReachability(
    pathConstraints=PathConstraints(startLocation="enter(as2border.*)"),
    headers=HeaderConstraints(dstIps="ofLocation(host.*)", srcIps = "0.0.0.0/0"),
).answer(
    snapshot=EX1_CHANGE_FIXED_NAME, 
    reference_snapshot=EX1_BASE_NAME)
display_html(answer.frame())

answer = bfq.differentialReachability(
    pathConstraints=PathConstraints(startLocation=".* - as2core.*"),
    headers=HeaderConstraints(dstIps="ofLocation(.* - as2core.*)"),
).answer(
    snapshot=EX1_CHANGE_FIXED_NAME, 
    reference_snapshot=EX1_BASE_NAME)
display_html(answer.frame())

,Flow,Base_Traces,Base_TraceCount,Delta_Traces,Delta_TraceCount


,Flow,Base_Traces,Base_TraceCount,Delta_Traces,Delta_TraceCount


Success! We have now verified that our change will correctly cost-out `as2core1` without affecting end-to-end network behavior. We are ready to deploy the change and do the maintenance work for `as2core1` with complete confidence. 

Let's recap the steps we took to verify this change:
1. First, we verified that the primary intent of the change is achieved: traffic is moved from `as2core1` to `as2core2`. We did this in two steps:
  1. Before the change, we checked that *some traffic* is routed through `as2core1` and *no traffic* is routed through `as2core2`.
  1. After the change, we checked that *no traffic* is routed through `as2core1` and *some traffic* is routed through `as2core2`.
1. Second, we 

## Example 2

Describe the network and the desired change: deny SSH from outside the AS.


## Step 1: Test current behavior

In [ ]:
EX2_NETWORK_NAME = "differential-example2"
EX2_BASE_NAME = "base"
EX2_BASE_PATH = "networks/differential-ex2-base"

bf_set_network(EX2_NETWORK_NAME)
bf_init_snapshot(EX2_BASE_PATH, name=EX2_BASE_NAME, overwrite=True)

In [ ]:
answer = bfq.reachability(
    pathConstraints = PathConstraints(startLocation="enter(as2border.*[GigabitEthernet0/0])", endLocation="host.*"),
    headers = HeaderConstraints(applications="SSH")
).answer(snapshot=EX2_BASE_NAME)
display_html(answer.frame())

These results show that SSH traffic can indeed currently reach the hosts from outside the AS.

## Step 2: Author the change


```
$ diff -r networks/differential-ex2-base networks/differential-ex2-change
diff -r networks/differential-ex2-base/configs/as2border1.cfg networks/differential-ex2-change/configs/as2border1.cfg
123a124
>  deny   tcp any 2.0.0.0 0.255.255.255 eq 22
```

In [ ]:
EX2_CHANGE_NAME = "change"
EX2_CHANGE_PATH = "networks/differential-ex2-change"

bf_init_snapshot(EX2_CHANGE_PATH, name=EX2_CHANGE_NAME, overwrite=True)

In [ ]:
answer = bfq.reachability(
    pathConstraints = PathConstraints(
        startLocation="enter(as2border.*[GigabitEthernet0/0])", 
        endLocation="host.*"),
    headers = HeaderConstraints(applications="SSH")
).answer(snapshot=EX2_CHANGE_NAME)
display_html(answer.frame())